In [ ]:
from common.common_imports import *
from sqlalchemy import create_engine, Table, Index, MetaData, select, distinct, exists
from sqlalchemy.sql import and_, desc
from data.h5py import H5PY_CONTROLLER
from data.history import HISTORY_DATA
from data.info import INFO_DATA

In [ ]:
class DB:
    def __init__(self) -> None:
        self.engine = create_engine('sqlite:////home/kanesweet/kanesweet/stonks.0.1/stonks.db')
        self.connection = self.engine.connect()
        self.metadata = MetaData(bind=self.engine)
        self.history_table = Table('history', self.metadata, autoload=True, autoload_with=self.engine)
        self.attached = False

db = DB()

In [ ]:
h5py = H5PY_CONTROLLER()
tickers_idx = pd.read_csv("data/csv/tickers_idx.csv") #7243
to_remove = []
for i, row in tickers_idx.iterrows():
    symbol = row["ticker"]
    
    query = (select(
                db.history_table.c.Date,
                db.history_table.c.Open,
                db.history_table.c.High,
                db.history_table.c.Low,
                db.history_table.c.Close,
                db.history_table.c.Volume,
            )
            .where(db.history_table.c.Ticker == symbol)
            .order_by(db.history_table.c.Date)
            )

    df = pd.read_sql(query, db.connection)
        
    # convert dates
    df['year'] = df["Date"].dt.year
    df['month'] = df["Date"].dt.month
    df['day'] = df["Date"].dt.day
    df = df.drop(columns=["Date"])

    array = df.to_numpy()

    # add broadcasted company info embeddings
    # info = INFO_DATA().info_array_from_index(HISTORY_DATA().ticker_stoi(symbol))
    # info = info.reshape(1, -1)
    # info = np.broadcast_to(info, (array.shape[0], info.shape[1]))
    # array = np.concatenate((array, info), axis=1)

    h5py.set_dataset(symbol+"_history", array)

In [ ]:
symbol = "AAPL"
array = h5py.get_dataset(symbol+"_history")
df = pd.DataFrame(array, columns=[
    'Open', 
    'High', 
    'Low', 
    'Close',
    'Volume', 
    'year',
    'month',
    'day'])
print(df)